# Creating Networks from TSV Tutorial

In this tutorial, we will show you how to create an NDEx network using a TSV file and a loading plan file in json that we will refer to as Loading Plan. In addition, we will cover some important files like delim2cx.py.

## Requirements

This tutorial requires:
* Python 2.7.9
* The latest version of the PIP Python package manager
* These packages for TSV Loader:
    * ndex
    * gspread
* Packages specific to this tutorial:
    * pandas
    

## Documents Used in this Tutorial

You need two documents to create a network that are covered in detail below. In this example, we will use two example documents: **idekerlab-1.txt and idekerlab-1-plan.json**. For your own purposes, you may change these documents. 

The TSV file, in this case idekerlab-1.txt, should be organized where information is separated by tabs. The first row contains the attribute titles, including the source node, source attributes, target node, target attributes, edge, and edge attributes. This format can then be converted to an excel file delimited by tabs. Below is a segment of idekerlab-1.txt as an example:

In [2]:
#Formatting delimited file as table
import pandas as pd
formatted = pd.read_csv('idekerlab-1.txt', sep='\t')
formatted[:10]

IOError: File idekerlab-1.txt does not exist

The loading plan file, in this case idekerlab-1-plan, is a json file that has to contain at least a source plan, a target plan, and a edge plan. These plans specify the interpretation of each column in the TSV file. Columns without a specified interpretation will be ignored.

Your plan must include the properties *source_plan, target_plan, and edge_plan*.*Nodemapping*  requires at least an *id_column or node_name_column*, but it could also contain the properties *id_prefix and property_columns*. Layered within *property_columns* is also a requirement to at least contain the *column_name or attribute_name* and possible properties such as *value_prefix, data_type, and default_value*.

Source and Target Plan:
* id_column: unique identifier for nodes in the network
* node_name_column: name for the node, if no identifier is specified, the name will be used as the identifier
* property_columns: column names to map to the specified node. Unspecified columns will be ignored.

Edge Plan:
* default_predicate: specifies the predicate (edge type) for edges unless explicity specified
* predicate: specifies the predicate (edge type) for the specified edge
* property_columns: column names to map to the specified edge. Unspecified columns will be ignored.

Below is a segment of idekerlab-1-plan.json as an example:

In [ ]:
{
    "source_plan": 
    {
        "id_column": "BAIT_OFFICIAL_SYMBOL",
        "node_name_column": "BAIT_OFFICIAL_SYMBOL",
        "property_columns": ["BAIT_GENE_ID"]
    },
    "target_plan": 
    {
        "id_column": "PREY_OFFICIAL_SYMBOL",
        "node_name_column": "PREY_OFFICIAL_SYMBOL",
        "property_columns": ["PREY_GENE_ID"]
    },
    "edge_plan": 
    {
        "default_predicate": "binds to",
        "property_columns": ["EXPERIMENTAL_SYSTEM"]
    }
}

## delim2cx.py

The delim2cx module has two main classes: TSVLoadingPlan and TSV2CXConverter.

### TSVLoadingPlan

The TSVLoadingPlan object contains the structure loaded from the loading plan file and checks it vs a json schema (from the internal file loading_plan_schema.json) to verify that it fulfills all requirements. 

### TSV2CXConverter

A TSV2CXConverter object is then created from the TSV file and the TSVLoadingPlan object. The TSV file is processed as the TSV2CXConverter is instantiated.

In [ ]:
import tsv.delim2cx as d2c
import ndex.client as nc
import ndex.beta.toolbox as toolbox
import ndex.beta.layouts as layouts
import ndex.networkn as networkn
import requests

In [ ]:
print "loading plan from: " + plan_name
import_plan = d2c.TSVLoadingPlan(plan_name)

In [ ]:
print "parsing tsv file using loading plan ..."
tsv_converter = d2c.TSV2CXConverter(import_plan)

In [ ]:
my_ndex = nc.Ndex("http://" + arg.server, arg.username, arg.password)

## create_network_from_tsv.py

This tutorial uses a python script as an example for the delim2cx module that uploads the network onto the dev2.ndexbio website.  

### Parameters

In order to upload a network using this script, there are at least seven script parameters necessary: 
* username
* password
* server
* tsv
* plan
* name of network
* description of network

Optional parameters are:
* template id
* layout
* uuid of network to update

### Overview

Once the parameters are loaded into the script properly, it will first attempt to parse the loading plan and then parse the TSV file using the loading plan. The way that the script does this is outlined in more detail in the section titled delim2cx.py. If there is a specified template that you inputted into the script parameters, the script will load that template. Otherwise, it will go with a default template. Then, it will upload the network onto the server using your username and password. 

### Loading Networks

This section will explain the syntax required to run this program in Jupyter Notebook. Running this program from the command line is very similar.

### Jupyter Notebook

The specific syntax is:

In [2]:
%run ./create_network_from_tsv.py janedoe janedoepass dev2.ndexbio.org idekerlab-1.txt idekerlab-1-plan.json Title_of_Network "Description of network"

loading plan from: idekerlab-1-plan.json
parsing tsv file using loading plan ...
Done.


Note that after the file name, each parameter that is necessary for the upload follows and is separated from each other by spaces. 